# Analysis of `floating-car-data.xml`

In [ ]:
import pprint, math, datetime, sys, time
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

from paths import normal, scenarios

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

file = f"{normal}/floating-car-data.xml"
s_file = f"{scenarios[0]}/floating-car-data.xml"

### Traffic counts for SUMO edges

In [ ]:
def count_vehicles_at_edges(file_path, edges, interval=60*60):
    class SAXParser:
        def __init__(self):
            self.counts = {}
            self.interval_counts = {}

        def start(self, tag, attr):
            if tag == "timestep":
                time = float(attr["time"])
                
                if time > 0 and time % interval == 0:
                    self.counts[time-interval] = self.interval_counts
                    self.interval_counts = {}

            if tag == "vehicle":
                vid = attr["id"]
                edge = attr["lane"].split("_")[0]

                if edge not in edges:
                    return

                if "cluster" in edge:
                    return

                if edge in self.interval_counts:
                    edge_counts = self.interval_counts[edge]
                    if vid not in edge_counts:
                        edge_counts.append(vid)
                        self.interval_counts[edge] = edge_counts
                else:
                    self.interval_counts[edge] = [vid]

        def end(self, tag):
            pass

        def close(self):
            pass

    sax_parser = SAXParser()
    xml_parser = etree.XMLParser(target=sax_parser)
    etree.parse(file_path, parser=xml_parser)

    return sax_parser.counts

matsim_count_edges = ["4067952", "375311156#0", "432205808#0"]

test_count_edges = ["627460044#0"]

interval = 60*60

vehicle_counts_per_edge = count_vehicles_at_edges(file, edges=test_count_edges, interval=interval)

hours = list(map(lambda i: i / interval, vehicle_counts_per_edge.keys()))

In [ ]:
for edge in test_count_edges:
    counts = []
    for edge_counts in vehicle_counts_per_edge.values():
        value = 0
        if edge in edge_counts:
            value = len(edge_counts[edge])

        counts.append(value)

    fig, ax = plt.subplots()
    ax.bar(hours, counts)
    ax.set_xticks(hours)
    ax.set_xlabel("Hour of the day", labelpad=40)
    ax.set_ylabel("No. of vehicles", labelpad=40)
    ax.set_title(f"Hourly traffic counts at edge {edge}")

### Traffic heatmap

In [ ]:
from pyproj import Proj, transform
from shapely.geometry import Point

external_crs = "EPSG:4326"
internal_crs = "EPSG:3068"

extProj = Proj(external_crs)
inProj = Proj(internal_crs)

bbox = [52.4828, 13.2136, 52.5330, 13.3594]

lat_min, long_min, lat_max, long_max = bbox

ymin, xmin = transform(extProj, inProj, lat_min, long_min)
ymax, xmax = transform(extProj, inProj, lat_max, long_max)

print(xmin, xmax)
print(ymin, ymax)

cellsize = 10

xgrid = np.linspace(xmin, xmax, int((xmax - xmin) / cellsize))
ygrid = np.linspace(ymin, ymax, int((ymax - ymin) / cellsize))

x_points = gpd.GeoDataFrame({"geometry": [Point(x, ygrid[0]) for x in xgrid]})
y_points = gpd.GeoDataFrame({"geometry": [Point(xgrid[0], y) for y in ygrid]})

x_points.crs = internal_crs
y_points.crs = internal_crs

long_points = x_points.to_crs(external_crs)
lat_points = y_points.to_crs(external_crs)

long_grid = np.array(long_points.geometry.x)
lat_grid = np.array(lat_points.geometry.y)

def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return array[idx]

class Done(Exception):
    pass

def count_vehicles_in_grid(file_path, start=0, end=math.inf):
    class SAXParser:
        def __init__(self):
            self.vehicles_per_cell = {}
            self.skip_timestep = False
            self.counts = np.zeros((len(lat_grid), len(long_grid)))

        def start(self, tag, attr):
            if tag == "timestep":
                self.skip_timestep = False
                
                t = float(attr["time"])
                
                if t < start:
                    self.skip_timestep = True
                    return
          

                if t > end:
                    raise Done()
                    return

            if tag == "vehicle":
                if self.skip_timestep:
                    return

                vid = attr["id"]
                long = float(attr["x"])
                lat = float(attr["y"])

                cell_long = find_nearest(long_grid, long)
                cell_lat = find_nearest(lat_grid, lat)
                cell_key = f"{cell_long},{cell_lat}"

                if cell_key in self.vehicles_per_cell:
                    if vid in self.vehicles_per_cell[cell_key]:
                        return

                i = np.nonzero(lat_grid == cell_lat)[0][0]
                j = np.nonzero(long_grid == cell_long)[0][0]
                self.counts[i][j] += 1

                if cell_key in self.vehicles_per_cell:
                    self.vehicles_per_cell[cell_key].append(vid)
                else:
                    self.vehicles_per_cell[cell_key] = [vid]

        def end(self, tag):
            pass

        def close(self):
            pass

    sax_parser = SAXParser()
    xml_parser = etree.XMLParser(target=sax_parser)

    try:
        etree.parse(file_path, parser=xml_parser)
    except Done:
        pass

    return sax_parser.counts

In [ ]:
counts = count_vehicles_in_grid(file)

fig, ax = plt.subplots()
img = ax.imshow(counts, origin="lower", cmap="viridis_r", extent=[xmin, xmax, ymin, ymax])
img.cmap.set_under("w")
img.set_clim(0.1)

In [ ]:
counts = count_vehicles_in_grid(s_file)

fig, ax = plt.subplots()
img = ax.imshow(counts, origin="lower", cmap="viridis_r", extent=[xmin, xmax, ymin, ymax])
img.cmap.set_under("w")
img.set_clim(0.1)